# Data Science Ex 09 - Clustering (k-Means & k-Medoids)

01.04.2021, Lukas Kretschmar (lukas.kretschmar@ost.ch)

## Let's have some Fun with k-Means and k-Medoids Clustering!

In this exercise, we are going to have a look at the k-Means and k-Medoids clustering approaches and clustering in general.
Since this topic is complex and it is important that you understand the concept of a cluster, we focus on simple examples in this exercise.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set()

## Introduction

### Updates & Installation

Before we can start with the exercises, we need to install a new package.
I'm aware that this is a bit risky since we are not at the same location, but we'll try it.
If you cannot update and install your Anaconda enviornment, you are not able to use the k-Medoids examples and exercises.

Open the *Anaconda Prompt* with advanced privileges (administrator rights) (Mac users just open a terminal) and enter the following command:

```bash
conda update --all
```

This is the same command that we also used in the first exercise session.
It will update your environment with the latests versions of all the packages you have already installed.
We need this, since otherwise the next command might wants to downgrade the majority your packages (what we don't want).

Now, we can install a new package that contains the *k-Medoids* algorithm.
Execute the command below in the same command prompt/terminal.

```bash
conda install -c conda-forge scikit-learn-extra
```

I recommend a restart of Anaconda, just to be sure.
So close the notebook and Anaconda, and open it again.

### Peparing the Data

First we need some data to use with our clustering algorithms.

In [ ]:
data = pd.read_csv("./Demo_5Clusters.csv")
data.head(5)

As you can see, we have 3 columns in our dataset.
`l` contains a number from 0 to 4 representing the cluster a point belongs to.
`x` and `y` are coordinates in a two dimensional space.
Hence, we can visualize the given data.

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
data.plot.scatter(ax=ax, x="x", y="y", c="l", s=50, cmap="rainbow", colorbar=False)

As we can see, the clusters are quite easily distinguishable.
Well, the two on the right are a bit close.

### Building Clusters

Now, we know that there are two simple clustering algorithms that we can use:
- k-Means
- k-Medoids

Both offer the same interface that we are already familiar with.
There is a `fit()` method to let the algorithm find the clusters.
And there is a `predict()` method to predict the clusters of points.
There is also a `fit_predict()` method that combines both calls.
This method comes in handy when we want to predict the clusters of the data used for finding them.

#### k-Means

Reference: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

In [ ]:
from sklearn.cluster import KMeans

Let's build the model, find the clusters and predict the clusters for the given points.
For now, we need to know the number of clusters we want to find.
Based on the provided data, we know that there are 5.

In [ ]:
model = KMeans(n_clusters=5, random_state=42) # Setting up the algorithm for 5 clusters
l_pred = model.fit_predict(data[["x", "y"]]) # Defining that we only want to use columns x and y

We can also get the center of each cluster.

In [ ]:
centers = pd.DataFrame(model.cluster_centers_, columns=["x", "y"])
centers

Have the results of the clustering algorithm, we can visualize again.

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
data.plot.scatter(ax=ax, x="x", y="y", c=l_pred, s=50, cmap="rainbow", colorbar=False) # Using the predict clusters for coloring (c=l_pred)
centers.plot.scatter(ax=ax, x="x", y="y", c="k", s=200, alpha=.5)

As you can see, the centers do not overlap with given points.
k-Means calculates its own points.

#### k-Medoid

Reference: https://scikit-learn-extra.readthedocs.io/en/latest/generated/sklearn_extra.cluster.KMedoids.html

*Note: k-Medoids is not an algorithm of scikit-learn, but an addition from another package.
It supports the same interface, though.*

In [ ]:
from sklearn_extra.cluster import KMedoids

Now, let's have a look how the k-Medoids algorithm solves the problem.

In [ ]:
model = KMedoids(n_clusters=5, random_state=42)
l_pred = model.fit_predict(data[["x", "y"]])

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
data.plot.scatter(ax=ax, x="x", y="y", c=l_pred, s=50, cmap="rainbow", colorbar=False)
centers = pd.DataFrame(model.cluster_centers_, columns=["x", "y"])
centers.plot.scatter(ax=ax, x="x", y="y", c="k", s=200, alpha=.5)

As you can see, now the center of each cluster is a given point within the cluster.

### Sometimes, it doesn't work that well

Let's have a look at another example.
The main difference between this and the previous one, now we only have 4 clusters that we want to find.

In [ ]:
data = pd.read_csv("./Demo_4Clusters.csv")
data.head(5)

As you can see, the data structure is the same.
You just find values in column `l` ranging from 0 to 3.

In [ ]:
model_means = KMeans(n_clusters=4, random_state=42)
l_pred_means = model_means.fit_predict(data[["x", "y"]])

model_medoids = KMedoids(n_clusters=4, random_state=42)
l_pred_medoids = model_medoids.fit_predict(data[["x", "y"]])

fig, ax = plt.subplots(1, 2, figsize=(20,10))
ax[0].set(title="k-Means")
data.plot.scatter(ax=ax[0], x="x", y="y", c=l_pred_means, s=50, cmap="rainbow", colorbar=False)
centers_means = pd.DataFrame(model_means.cluster_centers_, columns=["x", "y"])
centers_means.plot.scatter(ax=ax[0], x="x", y="y", c="k", s=200, alpha=.5)

ax[1].set(title="k-Medoids")
data.plot.scatter(ax=ax[1], x="x", y="y", c=l_pred_medoids, s=50, cmap="rainbow", colorbar=False)
centers_medoids = pd.DataFrame(model_medoids.cluster_centers_, columns=["x", "y"])
centers_medoids.plot.scatter(ax=ax[1], x="x", y="y", c="k", s=200, alpha=.5)

So, what happend here?
It seems that the k-Means algorithm was able to predict the clusters well.
But k-Medoids has its problems.

This has something to do how the initial cluster centers are defined.
While k-Means uses an optimized selection approach, k-Medoids selects them randomly.
And this may lead (as in this case) to strange results.

The optimized approach used by k-Means is called *k-means++* and is provied by the *init* hyperparameter.

In [ ]:
KMeans().get_params()

And there is also an optimized selection approach implemented for k-Medoids - called *k-medoids++*, it's just not the default.

In [ ]:
KMedoids().get_params()

So, if we run the same example again, but now with the optimized selection method, the results are as expected.

In [ ]:
model_medoids = KMedoids(n_clusters=4, init="k-medoids++", random_state=42)
l_pred_medoids = model_medoids.fit_predict(data[["x", "y"]])

fig, ax = plt.subplots(figsize=(10,10))

ax.set(title="k-Medoids")
data.plot.scatter(ax=ax, x="x", y="y", c=l_pred_medoids, s=50, cmap="rainbow", colorbar=False)
centers_medoids = pd.DataFrame(model_medoids.cluster_centers_, columns=["x", "y"])
centers_medoids.plot.scatter(ax=ax, x="x", y="y", c="k", s=200, alpha=.5)

### Number of Clusters

As you saw in the examples above, we always had to know the number of clusters we expect in the data.
And if this number is unknown, there is no way around trail & error.

The simplest method that we can use when we need to know the number of clusters, we can use the so called **elbow-method**.
Here, we just check, when the sum of squares of distances between the points and their cluster centers does not decrease anymore significantly.

In [ ]:
data = pd.read_csv("./Demo_FindingClusters.csv")
data.head(5)

Having the data, we just test a range of clusters.
The distance (square from points to centers) is held in the model and can be access with the `intertia_` property.

In [ ]:
sqr_dist = [] # Empty list for distances
clust = range(1,10)
for i in clust: # Testing numbers of clusters 1 to 9
    model = KMeans(n_clusters=i, random_state=42).fit(data)
    sqr_dist.append(model.inertia_)

Having stored the values, we can now simply plot a line.

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
ax.plot(clust, sqr_dist, "o-")
ax.set(xlabel="Number of Clusters", ylabel="Sum of Distances in Clusters")

And as you can see, after 4 clusters there is not much improvement.
So in this case, our data can likely be split into 4 different clusters.
And if we have a look at the data, we see that this is the case.

In [ ]:
model = KMeans(n_clusters=4, random_state=42)
l_pred = model.fit_predict(data)

fig, ax = plt.subplots(figsize=(10,10))
data.plot.scatter(ax=ax, x="x", y="y", c=l_pred, s=50, cmap="rainbow", colorbar=False)
centers = pd.DataFrame(model.cluster_centers_, columns=["x", "y"])
centers.plot.scatter(ax=ax, x="x", y="y", c="k", s=200, alpha=.5)

Thus, the elbow-method is one simple approach to determine the number of clusters in a dataset.
As you can imagine, in reality it's a bit harder.

### Showing the Cluster Boundaries

It could be also interesting to show the boundaries of your clusters.
The concept to show the boundaries is called a [Voronoi diagram](https://en.wikipedia.org/wiki/Voronoi_diagram).
A Voronoi diagram contains of points and perpendicular lines between two points.
If we take the centers of your clusters as our points, the lines of a Voronoi diagram are our cluster boundaries.

The scipy package offers the abbility to calculate these Voronoi diagrams and plot the in 2D.

In [ ]:
from scipy.spatial import Voronoi, voronoi_plot_2d

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
data.plot.scatter(ax=ax, x="x", y="y", c=l_pred, s=50, cmap="rainbow", colorbar=False)
centers = pd.DataFrame(model.cluster_centers_, columns=["x", "y"])
centers.plot.scatter(ax=ax, x="x", y="y", c="k", s=200, alpha=.5)

# Voronoi
voronoi = Voronoi(centers)
f = voronoi_plot_2d(voronoi, ax=ax, show_points=False, show_vertices=False) # show_points=False does not show centers (we do that already), show_vertices=False does not show points where lines are connected.
# The method returns the figure assigned to the axis ax. If we don't store it in a variable, Jupyter will plot the figure twice.

### What if the points are not in a Sphere?

What we have seen until now, are simple examples of points that we can easily fit into clusters.
And k-Means and k-Medoids algorithms work good on such datasets.
But as soon as we have other constructs, that we as humans easily can cluster, these algorithms fail miserably.

In [ ]:
data = pd.read_csv("./Demo_NonSpherical.csv")
data.head(5)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
data.plot.scatter(ax=ax, x="x", y="y", c="l", s=50, cmap="rainbow", colorbar=False)

If we use our algorithms on this dataset, the results aren't quite good.

In [ ]:
model_means = KMeans(n_clusters=2, random_state=42)
l_pred_means = model_means.fit_predict(data[["x", "y"]])
centers_means = pd.DataFrame(model_means.cluster_centers_, columns=["x", "y"])

model_medoids = KMedoids(n_clusters=2, init="k-medoids++", random_state=42)
l_pred_medoids = model_medoids.fit_predict(data[["x", "y"]])
centers_medoids = pd.DataFrame(model_medoids.cluster_centers_, columns=["x", "y"])

fig, ax = plt.subplots(1,2, figsize=(20,10))
ax[0].set(title="k-Means")
data.plot.scatter(ax=ax[0], x="x", y="y", c=l_pred_means, s=50, cmap="rainbow", colorbar=False)
centers_means.plot.scatter(ax=ax[0], x="x", y="y", c="k", s=200, alpha=.5)

ax[1].set(title="k-Medoids")
data.plot.scatter(ax=ax[1], x="x", y="y", c=l_pred_medoids, s=50, cmap="rainbow", colorbar=False)
centers_medoids.plot.scatter(ax=ax[1], x="x", y="y", c="k", s=200, alpha=.5)

As you can see, the algorithms work as expected but with such a form they cannot be successful.

Luckily, there is a special clustering algorithm that uses internally the k-Means algorithm, but can handle these structures.
The algorithm is called `SpectralClustering` and leverages the *nearest neighbors* approach to choose points of the cluster.

In [ ]:
from sklearn.cluster import SpectralClustering
model = SpectralClustering(n_clusters=2, affinity="nearest_neighbors", random_state=42)
l_pred = model.fit_predict(data[["x", "y"]])

fig, ax = plt.subplots(figsize=(10,10))
data.plot.scatter(ax=ax, x="x", y="y", c=l_pred, s=50, cmap="rainbow", colorbar=False)
# The generated warning can be ignored.

As you can see, both clusters are found as expected.
So, it's really important that you have an understanding of how the data might look like.
Thus, clustering is a hard task and will take some time.

With this basic introdction, you are now ready to head into the exercises.
You will concentrate on what you have learned today.
And in the next exercise, we will introduced some advanced methods and more complex use cases.

### Nice2Know: 3D

Depending on the use case or what we want to visualize, it is also possible to expand into the 3rd dimension.

In [ ]:
data3d = pd.read_csv("./Demo_5Clusters_3D.csv")
data3d.head(5)

We have again a dataset with 5 clusters but now 3 values per point.
Running the clustering algorithms stays the same (and is the same for any higher dimension - having more features - datasets).

In [ ]:
model3d = KMeans(n_clusters=5, random_state=42)
l_pred_3d = model3d.fit_predict(data3d[["x", "y", "z"]])
centers3d = pd.DataFrame(model3d.cluster_centers_, columns=["x", "y", "z"])

To use 3D visualization, we need a new package that extends matplotlib to support 3-dimensional data.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

To enable 3D within matplotlib, we have to provide an additional parameter (`subplot_kw`) that has a key-value pair of `"projection": "3d"`.

In [ ]:
fig, ax = plt.subplots(figsize=(10,10), subplot_kw={"projection": "3d"})
ax.scatter(data3d["x"], data3d["y"], data3d["z"], c=l_pred_3d, s=50, cmap="rainbow", alpha=.25)
ax.scatter(centers3d["x"], centers3d["y"], centers3d["z"], c="k", s=200)
ax.set(xlabel="x", ylabel="y", zlabel="z")

We can also take different perspectives to get a better idea where the clusters are located.

In [ ]:
def scatter3D(data, centers, clusters, ax, title):
    ax.scatter(data["x"], data["y"], data["z"], c=clusters, s=50, cmap="rainbow", alpha=.25)
    ax.scatter(centers["x"], centers["y"], centers["z"], c="k", s=200)
    ax.set(xlabel="x", ylabel="y", zlabel="z", title=title)
    return ax
    
fig, ax = plt.subplots(2,2,figsize=(20,20), subplot_kw={"projection":"3d"})

scatter3D(data3d, centers3d, l_pred_3d, ax[0,0], "Overview").view_init(10, 45) # view_init(elevation, horizontal angle) defines the perspective
scatter3D(data3d, centers3d, l_pred_3d, ax[0,1], "Y vs Z").view_init(10, 90)
scatter3D(data3d, centers3d, l_pred_3d, ax[1,0], "Y vs Z").view_init(10, 0)
scatter3D(data3d, centers3d, l_pred_3d, ax[1,1], "X vs Y").view_init(90, 45)

As you can see, it can be quite useful to show clusters in 3-dimensions.
And it's not that hard to achive that.

## Exercises

### Ex01 - Simple Clustering

In this exercise, you are going to use the k-Means and k-Medoids algorithms on a given dataset.
Thus, load **Ex09_01_Data.csv**.

How many points and clusters are in the dataset?
You don't have to run the algorithms for that.

Correct, there are 8 clusters.

Create a k-Means algorithm model and train it with the given data.

Where are the centers?

Predict the clusters for the same points you used for training.

Visualize the data and cluster centers.

**Bonus:** Try to plot the cluster boundaries.

Now do the same with the k-Medoids algorithm.

**Bonus:** Try to plot the cluster boundaries.

#### Solution

In [ ]:
# %load ./Ex09_01_Sol.py

### Ex02 - Finding the Number of Clusters

In this exercise, you are going to find the number of clusters for a given dataset.
You find the dataset in **Ex09_02_Data.csv**.

As you can see, the data is 3-dimensional.
But that shouldn't bother you.

Create a k-Means model and train it with the data.
Assume there is just 1 cluster.

What's the sum of all distances (*hint:* `inertia_`) to the center.

Now, do the same but with the assumption of 3 clusters.
What's now the sum of all distances?

Now, how about 8 clusters?
What's the sum of distances now?

Now, let's make it easier.
Use the elbow-method.
Check all numbers of clusters from 1 to 12.
And plot the line.

How many clusters do you expect in the data?
Predict the clusters for the given data.
Where are the cluster centers?

Let's plot the data.
Although, we have 3-dimensional data, we just plot 2 sides at the same time (so no 3D plot for now).
But plot all 3 perspectives into the same figure.

**Challenge:** Try to plot the cluster boundaries per view.

#### Solution

In [ ]:
# %load ./Ex09_02_Sol.py

### Ex03 - Area Usage Clustering

Load the data of **Ex09_03_Data.csv**.

In this file, you'll find all villages and cities in Switzerland with their corresponding coordinates (`E` & `N`) and the following features:
- `Pop` : Population
- `Pop_Density` : Population density
- `Area_Agri` : Percentage of the area assigned to agricultural usage
- `Area_Live` : Percentage of the area assigned to residential usage

All features are already normalized to the range of `[0,1]`.

In this exercise, we need the features at several steps.
To simplify the work, we define these features in a separate list to avoid retyping them every time we need them.
Thus, create a list containg the agricultural usage, residential usage, population density and population.

*Note: You can skip this step when you want to retype the features all the time.*

Visualize a comparison of the agricultural area usage and the residential area usage in a scatter plot.
Use the population density for coloring and the population for the size.

*Note:* You may have to increase the population indicator by a factor so your points will have a decent size.

Based on this simple visualization, we see that the area usages are entangled.
And the the population density and population correlates with the availability of space for residential usage.

Now, find a good number of posible clusters we could use to segment all cities and villages.

There are several good answers.
Something between `5` and `10` seems fair.

Use the `KMeans` algorithm, create a model and run it on the given features.

Recreate the scatter plot from above, but this time use the clusters for coloring.
And visualize the cluster centers.

We have now an indicator which cities and villages are comparable to eachother based on our defined feautres.
On the top right, we have cities that are large and have a high usage of residential area.
Below them, you have the agglomeration.
And at the bottom, we have several clusters of areas with low residential area usage and a varying availability of agricultural land usage.

Lastly, let's visualize the points in context of Switzerland.
Do the following:

1. Create a scatter plot using `E` and `N` for the coordinates.
2. Use the clusters for coloring the cities and villages.
3. Highlight `Rapperswil-Jona` to see to what cluster it belongs to.
4. Highlight your hometown (or another place).

All should be shown in one figure.
But it might be a good idea if you create it step by step.

#### Solution

In [ ]:
# %load ./Ex09_03_Sol.py

### Ex04 - 3D Clustering

This one is pretty easy.
You just have to plot 3D clusters.
The dataset to use is in **Ex09_04_Data.csv**.

Use the elbow-method to get the number of expected clusters.

Create the k-Means algorithm model for the expected number of clusters and predict the clusters.

Where are the centers?

Plot the cluster in 3D.

#### Solution

In [ ]:
# %load ./Ex09_04_Sol.py